In [1]:
import pandas as pd, numpy as np
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, roc_curve, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import time


2022-11-15 21:54:07.934330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_df = pd.read_csv("data/twitter_data_train_multiclass.csv")
test_df = pd.read_csv("data/twitter_data_test_multiclass.csv")

print(train_df.shape)
print(test_df.shape)

(9446, 238)
(1667, 238)


In [3]:
to_drop = [
    'Unnamed: 0',
    'account_type', 
    'account_type_multi',
    'id', 
    'name', 
    'screen_name',
    'profile_image_url',
    'protected',
    'verified',
    'description',
    'description_processed',
    'tweets_list',
    'tweets_list_processed',
    'url',
    'profile_use_background_image', 
    'profile_background_tile'
    ]

scaler = MinMaxScaler() 
train_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']] = scaler.fit_transform(train_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']])
test_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']] = scaler.transform(test_df[['statuses_count', 'favourites_count', 'followers_count', 'friends_count', 'number_of_mentions', 'listed_count', 'number_of_tags']])

X_train, y_train = train_df.drop(to_drop, axis=1), train_df['account_type_multi']
X_test, y_test = test_df.drop(to_drop, axis=1), test_df['account_type_multi']

# validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15)

In [4]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat  = to_categorical(y_test)

In [11]:
start_time = time.time()

model = Sequential()
model.add(Dense(64, activation = 'relu', input_dim = 222))
model.add(Dropout(.1))
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.fit(X_train, y_train_cat, epochs=20, validation_data=(X_val, y_val_cat))

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                14272     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 128)               8320      
                                                                 
 dense_11 (Dense)            (None, 4)                 516       
                                                                 
Total params: 23,108
Trainable params: 23,108
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
251/251 [==============================] - 1s 3ms/step - loss: 0.5005 - acc: 0.8353 - val_loss: 0.2180 - val_acc: 0.9224
Epoch 2/20
251/251 [==============================

Train

In [12]:
best_clf = model

start_time = time.time()

y_pred_train = best_clf.predict(X_train)
y_pred_train = np.argmax(y_pred_train,axis=1)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_train, y_pred_train, digits=5))
# roc_auc_score(y_test, y_pred_test)
print("F1 weighted: ", f1_score(y_train, y_pred_train, average='weighted'))

251/251 [==============================] - 0s 823us/step
Total time taken for the program execution 0.39833712577819824
              precision    recall  f1-score   support

           0    0.97385   0.99747   0.98552      1979
           1    0.99693   0.99085   0.99388      2295
           2    0.99760   0.99127   0.99443      1260
           3    1.00000   0.98958   0.99476      2495

    accuracy                        0.99215      8029
   macro avg    0.99210   0.99229   0.99215      8029
weighted avg    0.99230   0.99215   0.99218      8029

F1 weighted:  0.9921801299502511


test

In [13]:
start_time = time.time()

y_pred_test = best_clf.predict(X_test)
y_pred_test = np.argmax(y_pred_test,axis=1)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_test, y_pred_test, digits=5))
# roc_auc_score(y_test, y_pred_test)
print("F1 weighted: ", f1_score(y_test, y_pred_test, average='weighted'))

53/53 [==============================] - 0s 2ms/step
Total time taken for the program execution 0.3230469226837158
              precision    recall  f1-score   support

           0    0.95047   0.97579   0.96296       413
           1    0.98482   0.98696   0.98588       460
           2    0.97112   0.97464   0.97288       276
           3    0.99406   0.96911   0.98143       518

    accuracy                        0.97660      1667
   macro avg    0.97512   0.97662   0.97579      1667
weighted avg    0.97691   0.97660   0.97667      1667

F1 weighted:  0.9766668267991423
